<a href="https://colab.research.google.com/github/MangalaPriyadharshini/MangalaPriyadharshini/blob/main/LSTMSongClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import librosa
import numpy as np
import shutil
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')


np.random.seed(42)
tf.random.set_seed(42)

# 1️⃣ ZIP EXTRACTION
def extract_zip(zip_path, extract_dir):
    if os.path.exists(extract_dir):
        shutil.rmtree(extract_dir)
    os.makedirs(extract_dir, exist_ok=True)

    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

        audio_files = []
        audio_extensions = ('.mp3', '.wav', '.flac', '.m4a', '.aac')
        for root, dirs, files in os.walk(extract_dir):
            for file in files:
                if file.lower().endswith(audio_extensions):
                    full_path = os.path.join(root, file)
                    audio_files.append(full_path)
                    print(f"🎵 Found audio file: {file}")

        print(f"📁 Total audio files found: {len(audio_files)}")
        return audio_files
    except Exception as e:
        print(f"❌ Error extracting zip: {str(e)}")
        return []

# 2️⃣ FEATURE EXTRACTION
def extract_audio_features(file_path, duration=30, target_length_features=None):
    try:
        y, sr = librosa.load(file_path, duration=duration)
        features = []

        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        features.append(float(tempo))

        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)
        features.append(float(np.mean(spectral_centroids)))

        rms = librosa.feature.rms(y=y)
        features.append(float(np.mean(rms)))

        zcr = librosa.feature.zero_crossing_rate(y)
        features.append(float(np.mean(zcr)))

        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        features.append(float(np.mean(rolloff)))

        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_means = np.mean(mfccs, axis=1)
        features.extend(mfcc_means.tolist())

        features = np.array(features, dtype=np.float32)

        if target_length_features is not None:
            if len(features) < target_length_features:
                features = np.pad(features, (0, target_length_features - len(features)))
            elif len(features) > target_length_features:
                features = features[:target_length_features]

        return features
    except Exception as e:
        print(f"❌ Error processing {os.path.basename(file_path)}: {str(e)}")
        return None

# 3️⃣ RULE-BASED FALLBACK
def predict_genre_from_features(core_features):
    if len(core_features) < 1:
        return "UNKNOWN", 0.0
    tempo = core_features[0]
    if tempo < 90:
        return "ROMANTIC", 0.5
    elif tempo < 110:
        return "MELODY", 0.5
    elif tempo < 130:
        return "POP", 0.5
    elif tempo < 150:
        return "ROCK", 0.5
    else:
        return "RAP", 0.5

# 4️⃣ TRAINING DATA CREATION
def create_training_data(num_features=26):
    X_train, y_train = [], []
    genres = ['RAP', 'ROCK', 'POP', 'ROMANTIC', 'MELODY']
    for genre in genres:
        for _ in range(50):
            if genre == 'RAP':
                features = [np.random.uniform(130, 160), np.random.uniform(3500, 6000),
                            np.random.uniform(0.05, 0.2), np.random.uniform(0.3, 0.6),
                            np.random.uniform(3000, 5000)]
            elif genre == 'ROCK':
                features = [np.random.uniform(110, 140), np.random.uniform(2500, 5000),
                            np.random.uniform(0.1, 0.3), np.random.uniform(0.4, 0.7),
                            np.random.uniform(2500, 4000)]
            elif genre == 'POP':
                features = [np.random.uniform(95, 125), np.random.uniform(2000, 4500),
                            np.random.uniform(0.1, 0.25), np.random.uniform(0.4, 0.6),
                            np.random.uniform(2000, 3500)]
            elif genre == 'ROMANTIC':
                features = [np.random.uniform(60, 90), np.random.uniform(1000, 3000),
                            np.random.uniform(0.05, 0.15), np.random.uniform(0.2, 0.5),
                            np.random.uniform(1000, 2500)]
            else:  # MELODY
                features = [np.random.uniform(80, 110), np.random.uniform(1800, 3500),
                            np.random.uniform(0.05, 0.2), np.random.uniform(0.3, 0.6),
                            np.random.uniform(1500, 3000)]
            while len(features) < num_features:
                features.append(np.random.uniform(-100, 100))
            X_train.append(features)
            y_train.append(genre)
    return np.array(X_train), np.array(y_train)

# 5️⃣ MAIN FUNCTION
def main():
    from google.colab import files
    print("📤 Please upload a ZIP file containing your songs...")
    uploaded = files.upload()
    if not uploaded:
        print("❌ No file uploaded!")
        return []

    zip_filename = next(iter(uploaded))
    if not zip_filename.lower().endswith(".zip"):
        print("❌ Uploaded file is not a ZIP file.")
        return []

    zip_path = f"/content/{zip_filename}"
    extract_dir = "/content/music_dataset"

    print("📦 Step 1: Extracting zip file and searching for audio files...")
    audio_files = extract_zip(zip_path, extract_dir)
    if not audio_files:
        print("❌ No audio files found.")
        return []

    print("\n🧠 Step 2: Preparing LSTM-based genre classification model...")
    X_train, y_train = create_training_data()
    num_features = X_train.shape[1]

    le = LabelEncoder()
    y_encoded = le.fit_transform(y_train)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    # LSTM expects 3D input: (samples, timesteps, features)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(num_features, 1), return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(le.classes_), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_scaled, y_encoded, epochs=20, batch_size=16, verbose=0)

    print("\n🔍 Step 3: Analyzing each song...")
    results = []
    for i, audio_file in enumerate(audio_files, 1):
        features = extract_audio_features(audio_file, target_length_features=num_features)
        if features is not None:
            rule_genre, rule_confidence = predict_genre_from_features(features[:6])

            features_scaled = scaler.transform([features])
            features_scaled = features_scaled.reshape((1, num_features, 1))

            ml_pred = model.predict(features_scaled, verbose=0)
            ml_genre = le.inverse_transform([np.argmax(ml_pred)])[0]
            ml_confidence = np.max(ml_pred)

            final_genre = ml_genre if ml_confidence > 0.7 else rule_genre
            final_confidence = max(rule_confidence, ml_confidence)

            print(f"\n{i}/{len(audio_files)}: 🎵 {os.path.basename(audio_file)}")
            print(f"   🎯 FINAL GENRE: {final_genre} ({final_confidence:.1%} confidence)")

            results.append({
                'filename': os.path.basename(audio_file),
                'genre': final_genre,
                'confidence': float(final_confidence)
            })
        else:
            print(f"\n{i}/{len(audio_files)}: 🎵 {os.path.basename(audio_file)}")
            print("   ❌ Failed to extract features")
    return results


print("🎵 Starting LSTM Music Genre Analysis...")
results = main()


🎵 Starting LSTM Music Genre Analysis...
📤 Please upload a ZIP file containing your songs...


Saving Songs.zip to Songs.zip
📦 Step 1: Extracting zip file and searching for audio files...
🎵 Found audio file: Guru (Tamil) - Aaruyirae Video  A.R. Rahman.mp3
🎵 Found audio file: Maamannan - Nenjame Nenjame Video  Udhayanidhi Stalin  Vadivelu  A.R Rahman.mp3
🎵 Found audio file: Varaha Nadhikarai (From Sangamam).mp3
🎵 Found audio file: Kadal - Nenjukkule Video  A.R. Rahman.mp3
🎵 Found audio file: Pariyerum Perumal  Potta Kaatil Poovasam Video Song  Kathir, Anandhi  Santhosh Narayanan.mp3
🎵 Found audio file: Raavanan - Usure Pogudhey Video  A.R. Rahman  Vikram, Aishwarya Rai.mp3
🎵 Found audio file: Un Kanney Aayiram Kadha Pesudhey  Pradeep Kumar  SuperSinger  - Kulam.mp3
🎵 Found audio file: Anju Vanna Poove (Reprise) - Video  Thug Life  Kamal Haasan  Mani Ratnam  STR  AR Rahman.mp3
🎵 Found audio file: The Romance Of Power Paandi - Venpani Malare ft. Dhanush [Lyric Video]  Power Paandi  Sean Roldan.mp3
🎵 Found audio file: Irudhi Suttru Video Songs  Ey Sandakaara Full Video SongR Madhava